### Coletando notícias do G1

In [3]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import csv

## access given link and return corresponding new
def access_link(link):
    texto = ''
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    paragrafos = soup.find_all(class_="content-text__container")
    for par in paragrafos:
        texto = texto + '\n' + par.getText()
    title = soup.find(class_="content-head__title")
    return title.getText(), texto

links = ['https://g1.globo.com/economia/tecnologia/noticia/2018/08/09/por-que-e-uma-boa-ideia-usar-papel-aluminio-para-melhorar-o-sinal-de-wi-fi.ghtml',
        'https://g1.globo.com/economia/tecnologia/blog/altieres-rohr/post/2018/08/09/eficacia-do-windows-defender-para-proteger-navegadores-web-e-duvidas-do-whatsapp-web-pacotao-de-seguranca.ghtml',
        'https://g1.globo.com/economia/tecnologia/noticia/2018/08/09/aplicativos-movimentam-mais-de-us-3-bi-no-brasil-por-ano-com-publicidade-em-celulares.ghtml',
        'https://g1.globo.com/politica/noticia/2018/08/09/como-esconder-anos-de-posts-no-facebook-em-3-cliques.ghtml']

news_data = []
for link in links:
    title, texto = access_link(link)
    news_data.append({'title': title, 'content': texto})

for news in news_data:
    print(news['title'], '\n\n', news['content'], '\n\n\n')

Por que é uma boa ideia usar papel alumínio para melhorar o sinal de wi-fi 

 
 Graças à nossa luta diária para ter um bom wi-fi em casa, ficou famosa a ideia de usar papel alumínio para direcionar o sinal de internet. Não é um truque novo, mas é bom saber que a ciência demonstrou que ele funciona. 
 Segundo pesquisadores da Universidade de Darthmouth, nos Estados Unidos, colocar papel alumínio ou uma lata em volta das antenas do roteador pode melhorar o sinal, reduzir interferências e aumentar a segurança da conexão. 
 "Isso melhora a eficiência da rede sem fio em edifícios, ao reduzir o impacto que paredes e divisões internas têm", dizem os engenheiros de Darthmouth. 
 Como funciona o truque do papel alumínio para melhorar o sinal de wi-fi?
 As antenas dos roteadores são geralmente omnidirecionais, ou seja, seu sinal se dispersa para todos os lados. 
 Uma barreira de papel alumínio colocada em torno delas faz com que o sinal se torne unidirecional, ou seja, vá apenas em um sentido. 


### Calculando estatísticas básicas de cada notícia coletada (news_data)

In [11]:
import sys
import json
import nltk



stop_words = nltk.corpus.stopwords.words('portuguese') + [
    '.',
    ',',
    '--',
    '\'s',
    '?',
    ')',
    '(',
    ':',
    '\'',
    '\'re',
    '"',
    '-',
    '}',
    '{',
    ]

for news in news_data:
    sentences = nltk.tokenize.sent_tokenize(news['content'])

    words = [w.lower() for sentence in sentences for w in
             nltk.tokenize.word_tokenize(sentence)]

    fdist = nltk.FreqDist(words)

    # Basic stats

    num_words = sum([i[1] for i in fdist.items()])
    num_unique_words = len(fdist.keys())

    # Hapaxes are words that appear only once

    num_hapaxes = len(fdist.hapaxes())

    top_10_words_sans_stop_words = [w for w in fdist.items() if w[0]
                                    not in stop_words][:10]

    print(news['title'])
    print('\tNum Sentences:'.ljust(25), len(sentences))
    print('\tNum Words:'.ljust(25), num_words)
    print('\tNum Unique Words:'.ljust(25), num_unique_words)
    print('\tNum Hapaxes:'.ljust(25), num_hapaxes)
    print('\tTop 10 Most Frequent Words (sans stop words):\n\t\t', \
            '\n\t\t'.join(['%s (%s)'
            % (w[0], w[1]) for w in top_10_words_sans_stop_words]))
    print()

Por que é uma boa ideia usar papel alumínio para melhorar o sinal de wi-fi
	Num Sentences:           15
	Num Words:               401
	Num Unique Words:        186
	Num Hapaxes:             130
	Top 10 Most Frequent Words (sans stop words):
		 graças (1)
		luta (1)
		diária (1)
		ter (4)
		bom (2)
		wi-fi (4)
		casa (3)
		ficou (1)
		famosa (1)
		ideia (1)

Eficácia do Windows Defender para proteger navegadores web e dúvidas do WhatsApp Web: pacotão de segurança
	Num Sentences:           45
	Num Words:               1043
	Num Unique Words:        402
	Num Hapaxes:             253
	Top 10 Most Frequent Words (sans stop words):
		 alguma (2)
		dúvida (3)
		sobre (4)
		segurança (4)
		informação (1)
		antivírus (6)
		invasões (1)
		cibercrime (1)
		roubo (1)
		dados (4)

Aplicativos movimentam mais de US$ 3 bi no Brasil por ano com publicidade em celulares
	Num Sentences:           7
	Num Words:               173
	Num Unique Words:        92
	Num Hapaxes:             58
	Top 10 Most Frequ

### Sumarizando as notícias coletadas

In [12]:
import sys
import json
import nltk
import numpy

N = 100  # Number of words to consider
CLUSTER_THRESHOLD = 5  # Distance between words to consider
TOP_SENTENCES = 5  # Number of sentences to return for a "top n" summary

# Approach taken from "The Automatic Creation of Literature Abstracts" by H.P. Luhn

def _score_sentences(sentences, important_words):
    scores = []
    sentence_idx = -1

    for s in [nltk.tokenize.word_tokenize(s) for s in sentences]:

        sentence_idx += 1
        word_idx = []

        # For each word in the word list...
        for w in important_words:
            try:
                # Compute an index for where any important words occur in the sentence

                word_idx.append(s.index(w))
            except (ValueError) as e: # w not in this particular sentence
                pass

        word_idx.sort()

        # It is possible that some sentences may not contain any important words at all
        if len(word_idx)== 0: continue

        # Using the word index, compute clusters by using a max distance threshold
        # for any two consecutive words

        clusters = []
        cluster = [word_idx[0]]
        i = 1
        while i < len(word_idx):
            if word_idx[i] - word_idx[i - 1] < CLUSTER_THRESHOLD:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster = [word_idx[i]]
            i += 1
        clusters.append(cluster)

        # Score each cluster. The max score for any given cluster is the score 
        # for the sentence

        max_cluster_score = 0
        for c in clusters:
            significant_words_in_cluster = len(c)
            total_words_in_cluster = c[-1] - c[0] + 1
            score = 1.0 * significant_words_in_cluster \
                * significant_words_in_cluster / total_words_in_cluster

            if score > max_cluster_score:
                max_cluster_score = score

        scores.append((sentence_idx, score))

    return scores

def summarize(txt):
    sentences = [s for s in nltk.tokenize.sent_tokenize(txt)]
    normalized_sentences = [s.lower() for s in sentences]

    words = [w.lower() for sentence in normalized_sentences for w in
             nltk.tokenize.word_tokenize(sentence)]

    fdist = nltk.FreqDist(words)

    top_n_words = [w[0] for w in fdist.items() 
            if w[0] not in nltk.corpus.stopwords.words('portuguese')][:N]

    scored_sentences = _score_sentences(normalized_sentences, top_n_words)

    # Summaization Approach 1:
    # Filter out non-significant sentences by using the average score plus a
    # fraction of the std dev as a filter

    avg = numpy.mean([s[1] for s in scored_sentences])
    std = numpy.std([s[1] for s in scored_sentences])
    mean_scored = [(sent_idx, score) for (sent_idx, score) in scored_sentences
                   if score > avg + 0.5 * std]

    # Summarization Approach 2:
    # Another approach would be to return only the top N ranked sentences

    top_n_scored = sorted(scored_sentences, key=lambda s: s[1])[-TOP_SENTENCES:]
    top_n_scored = sorted(top_n_scored, key=lambda s: s[0])

    # Decorate the post object with summaries

    return dict(top_n_summary=[sentences[idx] for (idx, score) in top_n_scored],
                mean_scored_summary=[sentences[idx] for (idx, score) in mean_scored])


if __name__ == '__main__':

    # Load in output from blogs_and_nlp__get_feed.py

    #BLOG_DATA = sys.argv[1]
    #blog_data = json.loads(open(BLOG_DATA).read())
    
    for post in news_data:
       
        post.update(summarize(post['content']))

        print(post['title'])
        print('-' * len(post['title']))
        print()
        print('-------------')
        print('Top N Summary')
        print('-------------')
        print(' '.join(post['top_n_summary']))
        print()
        print('-------------------')
        print('Mean Scored Summary')
        print('-------------------')
        print(' '.join(post['mean_scored_summary']))
        print()

Por que é uma boa ideia usar papel alumínio para melhorar o sinal de wi-fi
--------------------------------------------------------------------------

-------------
Top N Summary
-------------

 Graças à nossa luta diária para ter um bom wi-fi em casa, ficou famosa a ideia de usar papel alumínio para direcionar o sinal de internet. Segundo pesquisadores da Universidade de Darthmouth, nos Estados Unidos, colocar papel alumínio ou uma lata em volta das antenas do roteador pode melhorar o sinal, reduzir interferências e aumentar a segurança da conexão. "Isso melhora a eficiência da rede sem fio em edifícios, ao reduzir o impacto que paredes e divisões internas têm", dizem os engenheiros de Darthmouth. Assim, você pode direcionar o wi-fi para a sala, por exemplo, o que fará com que outros ambientes fiquem sem sinal, mas ele estará concentrado onde pode ser mais necessário. Impedir que o sinal se propague em determinada direção e chegue apenas a alguns locais pode ter outros benefícios.

--

### Visualização HTML da sumarização

In [13]:
import os
import sys
import json
import nltk
import numpy

HTML_TEMPLATE = """<html>
    <head>
        <title>%s</title>
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    </head>
    <body>%s</body>
</html>"""

if __name__ == '__main__':

    # Marked up version can be written out to disk

    if not os.path.isdir('out/summarize'):
        os.makedirs('out/summarize')

    for post in news_data:
       
        post.update(summarize(post['content']))

        for summary_type in ['top_n_summary', 'mean_scored_summary']:
            post[summary_type + '_marked_up'] = '<p>%s</p>' % (post['content'], )
            for s in post[summary_type]:
                post[summary_type + '_marked_up'] = \
                post[summary_type + '_marked_up'].replace(s, '<strong>%s</strong>' % (s, ))

            filename = post['title'] + '.summary.' + summary_type + '.html'
            f = open(os.path.join('out', 'summarize', filename), 'w')
            html = HTML_TEMPLATE % (post['title'] + ' Summary', post[summary_type + '_marked_up'],)
            #f.write(str(html.encode('utf-8')))
            f.write(html)
            f.close()

            print("Data written to", f.name)

Data written to out\summarize\Por que é uma boa ideia usar papel alumínio para melhorar o sinal de wi-fi.summary.top_n_summary.html
Data written to out\summarize\Por que é uma boa ideia usar papel alumínio para melhorar o sinal de wi-fi.summary.mean_scored_summary.html
Data written to out\summarize\Eficácia do Windows Defender para proteger navegadores web e dúvidas do WhatsApp Web: pacotão de segurança.summary.top_n_summary.html
Data written to out\summarize\Eficácia do Windows Defender para proteger navegadores web e dúvidas do WhatsApp Web: pacotão de segurança.summary.mean_scored_summary.html
Data written to out\summarize\Aplicativos movimentam mais de US$ 3 bi no Brasil por ano com publicidade em celulares.summary.top_n_summary.html
Data written to out\summarize\Aplicativos movimentam mais de US$ 3 bi no Brasil por ano com publicidade em celulares.summary.mean_scored_summary.html
Data written to out\summarize\Como esconder anos de posts no Facebook em 3 cliques.summary.top_n_summa